In [1]:
import syft as sy
from syft.spdz import spdz
from syft.mpc.securenn import decompose, select_shares, private_compare, generate_zero_shares_communication, msb, relu_deriv
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor, _SPDZTensor, _SNNTensor
from syft.core.frameworks.torch import utils as torch_utils

import unittest
import numpy as np
import torch
import importlib

global display_chain

class Chain():
    def __init__(self, leaf=False):
        if not leaf:
            self.tensor = Chain(True)
            self.var = Chain(True)

display_chain = Chain()

display_chain.tensor.local = 'FloatTensor > _LocalTensor'

display_chain.tensor.pointer = 'FloatTensor > _PointerTensor'

display_chain.tensor.fixp_local = 'FloatTensor > _FixedPrecisionTensor > LongTensor > _LocalTensor'

display_chain.tensor.fixp_mpc_gpt = 'FloatTensor > _FixedPrecisionTensor > LongTensor > _SNNTensor > LongTensor > _GeneralizedPointerTensor'

display_chain.var.local = 'Variable > _LocalTensor\n' \
                          ' - FloatTensor > _LocalTensor\n' \
                          ' - - Variable > _LocalTensor\n' \
                          '   - FloatTensor > _LocalTensor'

display_chain.var.pointer = 'Variable > _PointerTensor\n' \
                            ' - FloatTensor > _PointerTensor\n' \
                            ' - - Variable > _PointerTensor\n' \
                            '   - FloatTensor > _PointerTensor'

display_chain.var.fixp_local = 'Variable > _FixedPrecisionTensor > Variable > _LocalTensor\n' \
                               ' - FloatTensor > _FixedPrecisionTensor > LongTensor > _LocalTensor\n' \
                               ' - - Variable > _FixedPrecisionTensor > Variable > _LocalTensor\n' \
                               '   - FloatTensor > _FixedPrecisionTensor > LongTensor > _LocalTensor'

display_chain.var.fixp_mpc_gpt = 'Variable > _FixedPrecisionTensor > Variable > _SNNTensor > Variable > _GeneralizedPointerTensor\n' \
                                 ' - FloatTensor > _FixedPrecisionTensor > LongTensor > _SNNTensor > LongTensor > _GeneralizedPointerTensor\n' \
                                 ' - - Variable > _FixedPrecisionTensor > Variable > _SNNTensor > Variable > _GeneralizedPointerTensor\n' \
                                 '   - FloatTensor > _FixedPrecisionTensor > LongTensor > _SNNTensor > LongTensor > _LocalTensor'


In [2]:
hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [3]:
L = spdz.Q_BITS

def random_as(tensor, mod=L):
    r = torch.LongTensor(tensor.get_shape()).random_(mod)
    return r

def xor(x, y):
    return x + y - 2 * x * y

def flip(x, dim):
    indices = torch.arange(x.get_shape()[dim] - 1, -1, -1).long()

    if hasattr(x.child, 'pointer_tensor_dict'):
        indices = indices.send(*list(x.child.pointer_tensor_dict.keys()))

    return x.index_select(dim, indices)

In [4]:
a = (torch.LongTensor([-1,3,-5,7])).share(alice,bob)
# d = relu_deriv(a)

In [5]:
type(a.child)

syft.core.frameworks.torch.tensor._SNNTensor

In [8]:
torch_utils.is_syft_tensor(type(a.child))

False

In [5]:
b = a.relu()

In [6]:
b.get()


 0
 3
 0
 7
[syft.core.frameworks.torch.tensor.LongTensor of size 4]

In [7]:
x = torch.FloatTensor([1.1, 2, 3])
x = x.send(bob).fix_precision().share(alice, bob)
assert torch_utils.chain_print(x, display=False) == display_chain.tensor.pointer
x_ = bob.get_obj(x.id_at_location).parent
assert torch_utils.chain_print(x_, display=False) == display_chain.tensor.fixp_mpc_gpt

z = x + x

x = x.get()

Exception: could not deserialize an object sent to router
{'___SNNTensor__': {'owner': 'bob', 'id': 131439295, 'shares': {'__LongTensor__': {'type': 'syft.core.frameworks.torch.tensor.LongTensor', 'torch_type': 'syft.LongTensor', 'data': [1100000, 2000000, 3000000], 'child': {'___GeneralizedPointerTensor__': {'owner': 'bob', 'id': 5323991053, 'pointer_tensor_dict': {'alice': {'___PointerTensor__': {'owner': 0, 'id': 6388814402, 'location': 'alice', 'id_at_location': 27111993731, 'torch_type': 'syft.LongTensor'}}, 'bob': {'___PointerTensor__': {'owner': 0, 'id': 9563565848, 'location': 'bob', 'id_at_location': 70111375023, 'torch_type': 'syft.LongTensor'}}}, 'torch_type': 'syft.LongTensor'}}}}, 'torch_type': 'syft.LongTensor'}}

In [8]:
sy._SyftTensor.__subclasses__()

[syft.core.frameworks.torch.tensor._LocalTensor,
 syft.core.frameworks.torch.tensor._WrapTorchObjectPlusIsMinusTensor,
 syft.core.frameworks.torch.tensor._PlusIsMinusTensor,
 syft.core.frameworks.torch.tensor._GeneralizedPointerTensor,
 syft.core.frameworks.torch.tensor._PointerTensor,
 syft.core.frameworks.torch.tensor._FixedPrecisionTensor,
 syft.core.frameworks.torch.tensor._SPDZTensor,
 syft.core.frameworks.torch.tensor._SNNTensor]

In [ ]:




assert torch_utils.chain_print(x, display=False) == display_chain.tensor.fixp_mpc_gpt

x = x.get()
assert torch_utils.chain_print(x, display=False) == display_chain.tensor.fixp_local

x = x.decode()
assert torch_utils.chain_print(x, display=False) == display_chain.tensor.local

z = z.get().get().decode()
assert torch.eq(z, torch.FloatTensor([2.2, 4, 6])).all()

In [21]:
def decompose(tensor):
    """
    decompose a tensor into its binary representation
    """
    powers = torch.arange(0,Q_BITS)
    for i in range(len(tensor.shape)):
        powers = powers.unsqueeze(0)
    tensor = tensor.unsqueeze(-1)
    moduli = 2 ** powers
    tensor = ((tensor+2**(Q_BITS)) / moduli.type_as(tensor)) % 2
    return tensor

def _pc_beta0(x, r):
    # note x and r are both binary tensors,
    # and dim -1 contains their bits
    # x should be shared, r should be public
    w = xor(x, r)
    z = r  - (x - 1)
    w_sum = torch.zeros(w.shape).type_as(w)
    for i in range(Q_BITS - 2, -1, -1):
        w_sum[..., i] = w[..., (i + 1):].sum(dim=-1, keepdim=True)
    c = z + w_sum
    return c


def _pc_beta1(x, t):
    w = xor(x, t)
    z = (x + 1) - t
    w_sum = torch.zeros(w.shape).type_as(w)
    for i in range(Q_BITS - 2, -1, -1):
        w_sum[..., i] = w[..., (i + 1):].sum(dim=-1, keepdim=True)
    c = z + w_sum
    return c

def _pc_else(workers, *sizes):

    u = generate_zero_shares_communication(*workers, *sizes)

    u_ptrdict = get_ptrdict(u)
    (w0, u0), (w1, u1) = u_ptrdict.items()
    u0 = u0.wrap(True)
    u1 = u1.wrap(True)

    c0 = u0 * 0
    c1 = u1 * 0
    for i in range(Q_BITS - 2, -1, -1):
        if i == 0:
            c0[..., i] = -1 * u0[..., i]
            c1[..., i] = u1[..., i]
        c0[..., i] = u0[..., i] + 1
        c1[..., i] = -1 * u1[..., i]
    ptr_dict = {w0:c0.child, w1:c1.child}
    c_gp = _GeneralizedPointerTensor(ptr_dict, torch_type='syft.LongTensor').wrap(True)
    c = _SPDZTensor(c_gp, torch_type='syft.LongTensor').wrap(True)
    return c

def generate_one_shares_communication(alice, bob, sizes):
    return torch.ones(sizes).long().share(alice, bob)

def xor(x, y):
    return x + y - 2 * x * y

def get_ptrdict(mpct):
    child = mpct
    while not isinstance(child, sy.core.frameworks.torch._GeneralizedPointerTensor) and child is not None:
        child = child.child
    if child is None:
        raise TypeError("Expected child tree to contain a GeneralizedPointerTensor")
    return child.pointer_tensor_dict

x = torch.LongTensor([5])#.share(bob, alice)

r = torch.LongTensor([4])

Q_BITS = spdz.Q_BITS

beta = (torch.rand(1)>0).long()

t = (r + 1) % (2 ** Q_BITS)

x_bits = decompose(x)
r_bits = decompose(r)
t_bits = decompose(t)

c_zeros = _pc_beta0(x_bits, r_bits)
c_ones = _pc_beta1(x_bits, t_bits)
c_other = _pc_else([alice, bob], x_bits.shape)

# TODO: recombine c properly here
c = torch.zeros(*x_bits.shape).long()
zeros = (beta == 0).long()
ones = (beta == 1).long()
others = (r == (2 ** Q_BITS - 1)).long()
ones = ones & (others - 1).abs()

gen sz torch.Size([1, 31])


In [22]:
c = (zeros * c_zeros) + (ones * c_ones)

In [ ]:
c[zeros] = c_zeros
c[ones] = c_ones
c[others] = c_other

In [75]:
c_ones



Columns 0 to 12 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 13 to 25 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 26 to 30 
    1     1     1     1     1
[syft.core.frameworks.torch.tensor.LongTensor of size 1x31]

In [17]:
def _pc_else(workers, *sizes):
    u = generate_zero_shares_communication(*workers, *sizes)
    (w0, u0), (w1, u1) = u.child.shares.child.pointer_tensor_dict.items()
    u0 = u0.wrap()
    u1 = u1.wrap()
    
    c0 = u0 * 0
    c1 = u1 * 0
    for i in range(Q_BITS - 2, -1, -1):
        if i == 0:
            c0[..., i] = -u0[..., i]
            c1[..., i] = u1[..., i]

        c0[..., i] = u0[..., i] + 1
        c1[..., i] = -u1[..., i]
    ptr_dict = {w0:c0.child, w1:c1.child}
    c_gp = _GeneralizedPointerTensor(ptr_dict, torch_type='syft.LongTensor').wrap(True)
    c = _SPDZTensor(c_gp, torch_type='syft.LongTensor').wrap(True)
    return c

In [21]:
c_other

[Head of chain]
[syft.core.frameworks.torch.tensor.LongTensor with no dimension]

In [66]:
def private_compare(x, r, beta, workers):
    """
    computes beta XOR (x > r)

    x is private input
    r is public input for comparison
    beta is public random bit tensor

    all of type _GeneralizedPointerTensor
    """
    t = (r + 1) % (2 ** Q_BITS)

    x_bits = decompose(x)
    r_bits = decompose(r)
    t_bits = decompose(t)

    zeros = beta == 0
    ones = beta == 1
    others = r == (2 ** Q_BITS - 1)
    ones = ones & (others - 1).abs()

    c_zeros = _pc_beta0(x_bits, r_bits)
    c_ones = _pc_beta1(x_bits, t_bits)
    c_other = _pc_else()

    # TODO: recombine c properly here
    # torch.zeros()
    c = torch.cat([c_zeros, c_ones, c_other], -1)

    s = random_as(c, mod=p)
    permute = torch.randperm(c.size(-1))
    d = s * c[..., permute]
    d.get()
    return (d == 0).max()